In [326]:
# import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [327]:
import sys, os 
sys.path.insert(0, os.getcwd()+"/../")

In [328]:
import sys, importlib

# Remove cached modules
for k in list(sys.modules.keys()):
    if k == "src.utils" or k.startswith("src.utils."):
        del sys.modules[k]

import src.utils as u
importlib.reload(u)

print("FILE:", u.__file__)
print("_OWNER_MAP exists?", hasattr(u, "_OWNER_MAP"))
print("OWNER names:", [x for x in dir(u) if "OWNER" in x])
print("OWNER_MAP value:", getattr(u, "_OWNER_MAP", None))


FILE: /research/cvl-lehoang2/project/CarPricePrediction/notebooks/../src/utils.py
_OWNER_MAP exists? True
OWNER names: ['_OWNER_MAP']
OWNER_MAP value: {'UnRegistered Car': 0, 'First': 1, 'Second': 2, 'Third': 3, 'Fourth': 4, '4 or More': 5}


In [329]:
from src.utils import parse_engine_cc, parse_value_and_rpm, _OWNER_MAP

In [330]:
# first 5 rows of dataset
car_data = pd.read_csv("../car_details_v4.csv")

car_data.head()

,Make,Model,Price,Year,Kilometer,Fuel Type,Transmission,Location,Color,Owner,Seller Type,Engine,Max Power,Max Torque,Drivetrain,Length,Width,Height,Seating Capacity,Fuel Tank Capacity
0,Honda,Amaze 1.2 VX i-VTEC,505000,2017,87150,Petrol,Manual,Pune,Grey,First,Corporate,1198 cc,87 bhp @ 6000 rpm,109 Nm @ 4500 rpm,FWD,3990.0,1680.0,1505.0,5.0,35.0
1,Maruti Suzuki,Swift DZire VDI,450000,2014,75000,Diesel,Manual,Ludhiana,White,Second,Individual,1248 cc,74 bhp @ 4000 rpm,190 Nm @ 2000 rpm,FWD,3995.0,1695.0,1555.0,5.0,42.0
2,Hyundai,i10 Magna 1.2 Kappa2,220000,2011,67000,Petrol,Manual,Lucknow,Maroon,First,Individual,1197 cc,79 bhp @ 6000 rpm,112.7619 Nm @ 4000 rpm,FWD,3585.0,1595.0,1550.0,5.0,35.0
3,Toyota,Glanza G,799000,2019,37500,Petrol,Manual,Mangalore,Red,First,Individual,1197 cc,82 bhp @ 6000 rpm,113 Nm @ 4200 rpm,FWD,3995.0,1745.0,1510.0,5.0,37.0
4,Toyota,Innova 2.4 VX 7 STR [2016-2020],1950000,2018,69000,Diesel,Manual,Mumbai,Grey,First,Individual,2393 cc,148 bhp @ 3400 rpm,343 Nm @ 1400 rpm,RWD,4735.0,1830.0,1795.0,7.0,55.0


In [331]:
car_data.describe()

,Price,Year,Kilometer,Length,Width,Height,Seating Capacity,Fuel Tank Capacity
count,2.059000e+03,2059.000000,2.059000e+03,1995.000000,1995.000000,1995.000000,1995.000000,1946.000000
mean,1.702992e+06,2016.425449,5.422471e+04,4280.860652,1767.991980,1591.735338,5.306266,52.002210
std,2.419881e+06,3.363564,5.736172e+04,442.458507,135.265825,136.073956,0.822170,15.110198
min,4.900000e+04,1988.000000,0.000000e+00,3099.000000,1475.000000,1165.000000,2.000000,15.000000
25%,4.849990e+05,2014.000000,2.900000e+04,3985.000000,1695.000000,1485.000000,5.000000,41.250000
50%,8.250000e+05,2017.000000,5.000000e+04,4370.000000,1770.000000,1545.000000,5.000000,50.000000
75%,1.925000e+06,2019.000000,7.200000e+04,4629.000000,1831.500000,1675.000000,5.000000,60.000000
max,3.500000e+07,2022.000000,2.000000e+06,5569.000000,2220.000000,1995.000000,8.000000,105.000000


In [332]:
car_data.columns

Index(['Make', 'Model', 'Price', 'Year', 'Kilometer', 'Fuel Type',
       'Transmission', 'Location', 'Color', 'Owner', 'Seller Type', 'Engine',
       'Max Power', 'Max Torque', 'Drivetrain', 'Length', 'Width', 'Height',
       'Seating Capacity', 'Fuel Tank Capacity'],
      dtype='object')

In [333]:
# fill in missing values

missing_cols = car_data.columns[car_data.isnull().any()]
for col in missing_cols:
    if car_data[col].dtype == 'object':
        car_data[col].fillna(car_data[col].mode()[0], inplace=True)
    else:
        car_data[col].fillna(car_data[col].median(), inplace=True)


/tmp/ipykernel_2356366/3779400965.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  car_data[col].fillna(car_data[col].mode()[0], inplace=True)
/tmp/ipykernel_2356366/3779400965.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inp

In [334]:
# Function to detect and handle outliers using IQR method
def handle_outliers_iqr(df, column):
    
    # Calculate first quartile (Q1 - 25th percentile)
    Q1 = df[column].quantile(0.25)
    
    # Calculate third quartile (Q3 - 75th percentile)
    Q3 = df[column].quantile(0.75)
    
    # Compute Interquartile Range (IQR)
    IQR = Q3 - Q1
    
    # Define lower and upper bounds for outliers
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Find all outlier rows
    outliers = df[
        (df[column] < lower_bound) | 
        (df[column] > upper_bound)
    ]
    
    # Print number of detected outliers
    print(f"Column {column}: Found {len(outliers)} outliers")

    # Handle outliers using capping (winsorization)
    # Instead of deleting, we clip extreme values to boundary values
    # This preserves information and avoids data loss
    df[column] = np.where(
        df[column] > upper_bound, upper_bound,
        np.where(df[column] < lower_bound, lower_bound, df[column])
    )
    
    return df


# Apply outlier handling to columns
# These columns were observed to have heavy outliers from histogram analysis
cols_outliers = ['Kilometer', 'Seating Capacity']

for col in cols_outliers:
    car_data = handle_outliers_iqr(car_data, col)

Column Kilometer: Found 36 outliers
Column Seating Capacity: Found 380 outliers


Categorical features

In [335]:
# display all the columns name
car_data.columns

Index(['Make', 'Model', 'Price', 'Year', 'Kilometer', 'Fuel Type',
       'Transmission', 'Location', 'Color', 'Owner', 'Seller Type', 'Engine',
       'Max Power', 'Max Torque', 'Drivetrain', 'Length', 'Width', 'Height',
       'Seating Capacity', 'Fuel Tank Capacity'],
      dtype='object')

In [336]:
df = car_data.drop(["Location", "Color"], axis=1)


In [337]:
# owner_map = {
#     "UnRegistered Car": 0,
#     "First": 1,
#     "Second": 2,
#     "Third": 3,
#     "Fourth": 4,
#     "4 or More": 5
# }

df["Owner_ord"] = df["Owner"].map(_OWNER_MAP)
df = df.drop("Owner", axis=1)


In [338]:
df["Transmission_bin"] = df["Transmission"].map({
    "Manual": 0,
    "Automatic": 1
})

df = df.drop("Transmission", axis=1)


In [339]:
onehot_cols = ["Fuel Type", "Seller Type", "Drivetrain"]

df = pd.get_dummies(df, columns=onehot_cols, drop_first=True)

# convert bool → 0/1
bool_cols = df.select_dtypes("bool").columns
df[bool_cols] = df[bool_cols].astype(int)


In [340]:
from sklearn.model_selection import KFold
import numpy as np

def cv_target_encode(df, col, target, n_splits=5):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    encoded = np.zeros(len(df))

    for train_idx, val_idx in kf.split(df):
        train_df = df.iloc[train_idx]
        val_df   = df.iloc[val_idx]

        means = train_df.groupby(col)[target].mean()
        encoded[val_idx] = val_df[col].map(means)

    return encoded


In [341]:
df["Model_te"] = cv_target_encode(df, "Model", "Price")
df["Make_te"]  = cv_target_encode(df, "Make", "Price")

# fill unseen categories
global_mean = df["Price"].mean()
df["Model_te"].fillna(global_mean, inplace=True)
df["Make_te"].fillna(global_mean, inplace=True)




/tmp/ipykernel_2356366/4148640344.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["Model_te"].fillna(global_mean, inplace=True)
/tmp/ipykernel_2356366/4148640344.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)'

In [342]:
# ===== Block A: Save inference-time Target Encoding artifacts (Make/Model -> mean price) =====
import joblib

# Inference-time TE maps (fit on FULL training data, not CV)
global_mean_price = float(df["Price"].mean())

make_te_map  = df.groupby("Make")["Price"].mean().to_dict()
model_te_map = df.groupby("Model")["Price"].mean().to_dict()

te_artifacts = {
    "global_mean_price": global_mean_price,
    "make_te_map": make_te_map,
    "model_te_map": model_te_map,
}

joblib.dump(te_artifacts, "te_artifacts.joblib")
print("Saved -> te_artifacts.joblib")
print("global_mean_price:", global_mean_price)
print("num makes:", len(make_te_map), "num models:", len(model_te_map))


Saved -> te_artifacts.joblib
global_mean_price: 1702991.6964545897
num makes: 33 num models: 1050


In [343]:
df = df.drop(["Model", "Make"], axis=1)

In [344]:
df.head()

,Price,Year,Kilometer,Engine,Max Power,Max Torque,Length,Width,Height,Seating Capacity,...,Fuel Type_LPG,Fuel Type_Petrol,Fuel Type_Petrol + CNG,Fuel Type_Petrol + LPG,Seller Type_Corporate,Seller Type_Individual,Drivetrain_FWD,Drivetrain_RWD,Model_te,Make_te
0,505000,2017,87150.0,1198 cc,87 bhp @ 6000 rpm,109 Nm @ 4500 rpm,3990.0,1680.0,1505.0,5.0,...,0,1,0,0,1,0,1,0,1.702992e+06,6.272787e+05
1,450000,2014,75000.0,1248 cc,74 bhp @ 4000 rpm,190 Nm @ 2000 rpm,3995.0,1695.0,1555.0,5.0,...,0,0,0,0,0,1,1,0,4.067778e+05,5.408743e+05
2,220000,2011,67000.0,1197 cc,79 bhp @ 6000 rpm,112.7619 Nm @ 4000 rpm,3585.0,1595.0,1550.0,5.0,...,0,1,0,0,0,1,1,0,3.035714e+05,7.011824e+05
3,799000,2019,37500.0,1197 cc,82 bhp @ 6000 rpm,113 Nm @ 4200 rpm,3995.0,1745.0,1510.0,5.0,...,0,1,0,0,0,1,1,0,7.750000e+05,1.785524e+06
4,1950000,2018,69000.0,2393 cc,148 bhp @ 3400 rpm,343 Nm @ 1400 rpm,4735.0,1830.0,1795.0,5.0,...,0,0,0,0,0,1,0,1,1.921500e+06,1.677177e+06


In [345]:
df.columns

Index(['Price', 'Year', 'Kilometer', 'Engine', 'Max Power', 'Max Torque',
       'Length', 'Width', 'Height', 'Seating Capacity', 'Fuel Tank Capacity',
       'Owner_ord', 'Transmission_bin', 'Fuel Type_CNG + CNG',
       'Fuel Type_Diesel', 'Fuel Type_Electric', 'Fuel Type_Hybrid',
       'Fuel Type_LPG', 'Fuel Type_Petrol', 'Fuel Type_Petrol + CNG',
       'Fuel Type_Petrol + LPG', 'Seller Type_Corporate',
       'Seller Type_Individual', 'Drivetrain_FWD', 'Drivetrain_RWD',
       'Model_te', 'Make_te'],
      dtype='object')

In [346]:
# cat_cols = [
#     "Make",
#     "Model",
#     "Fuel Type",
#     "Transmission",
#     "Location",
#     "Color",
#     "Owner",
#     "Seller Type",
#     "Drivetrain"
# ]


In [347]:
# for col in cat_cols:
#     unique_vals = car_data[col].unique()
    
#     print("=" * 50)
#     print(f"Column: {col}")
#     print(f"Number of unique values: {len(unique_vals)}")
#     print("Unique values:")
#     print(unique_vals)


In [348]:
car_data.head()

,Make,Model,Price,Year,Kilometer,Fuel Type,Transmission,Location,Color,Owner,Seller Type,Engine,Max Power,Max Torque,Drivetrain,Length,Width,Height,Seating Capacity,Fuel Tank Capacity
0,Honda,Amaze 1.2 VX i-VTEC,505000,2017,87150.0,Petrol,Manual,Pune,Grey,First,Corporate,1198 cc,87 bhp @ 6000 rpm,109 Nm @ 4500 rpm,FWD,3990.0,1680.0,1505.0,5.0,35.0
1,Maruti Suzuki,Swift DZire VDI,450000,2014,75000.0,Diesel,Manual,Ludhiana,White,Second,Individual,1248 cc,74 bhp @ 4000 rpm,190 Nm @ 2000 rpm,FWD,3995.0,1695.0,1555.0,5.0,42.0
2,Hyundai,i10 Magna 1.2 Kappa2,220000,2011,67000.0,Petrol,Manual,Lucknow,Maroon,First,Individual,1197 cc,79 bhp @ 6000 rpm,112.7619 Nm @ 4000 rpm,FWD,3585.0,1595.0,1550.0,5.0,35.0
3,Toyota,Glanza G,799000,2019,37500.0,Petrol,Manual,Mangalore,Red,First,Individual,1197 cc,82 bhp @ 6000 rpm,113 Nm @ 4200 rpm,FWD,3995.0,1745.0,1510.0,5.0,37.0
4,Toyota,Innova 2.4 VX 7 STR [2016-2020],1950000,2018,69000.0,Diesel,Manual,Mumbai,Grey,First,Individual,2393 cc,148 bhp @ 3400 rpm,343 Nm @ 1400 rpm,RWD,4735.0,1830.0,1795.0,5.0,55.0


In [349]:
# car_data = car_data.drop(columns=["Model"])
# car_data = car_data.drop(columns=["Make"])

car_data = car_data.drop(columns=["Model"])


In [350]:
car_data.head()

,Make,Model,Price,Year,Kilometer,Fuel Type,Transmission,Location,Color,Owner,Seller Type,Engine,Max Power,Max Torque,Drivetrain,Length,Width,Height,Seating Capacity,Fuel Tank Capacity
0,Honda,Amaze 1.2 VX i-VTEC,505000,2017,87150.0,Petrol,Manual,Pune,Grey,First,Corporate,1198 cc,87 bhp @ 6000 rpm,109 Nm @ 4500 rpm,FWD,3990.0,1680.0,1505.0,5.0,35.0
1,Maruti Suzuki,Swift DZire VDI,450000,2014,75000.0,Diesel,Manual,Ludhiana,White,Second,Individual,1248 cc,74 bhp @ 4000 rpm,190 Nm @ 2000 rpm,FWD,3995.0,1695.0,1555.0,5.0,42.0
2,Hyundai,i10 Magna 1.2 Kappa2,220000,2011,67000.0,Petrol,Manual,Lucknow,Maroon,First,Individual,1197 cc,79 bhp @ 6000 rpm,112.7619 Nm @ 4000 rpm,FWD,3585.0,1595.0,1550.0,5.0,35.0
3,Toyota,Glanza G,799000,2019,37500.0,Petrol,Manual,Mangalore,Red,First,Individual,1197 cc,82 bhp @ 6000 rpm,113 Nm @ 4200 rpm,FWD,3995.0,1745.0,1510.0,5.0,37.0
4,Toyota,Innova 2.4 VX 7 STR [2016-2020],1950000,2018,69000.0,Diesel,Manual,Mumbai,Grey,First,Individual,2393 cc,148 bhp @ 3400 rpm,343 Nm @ 1400 rpm,RWD,4735.0,1830.0,1795.0,5.0,55.0


In [351]:
# One-hot encoding
# df = pd.get_dummies(car_data, columns=cat_cols, drop_first=True)

In [352]:
# first 5 rows after preprocessing
df.head()

,Price,Year,Kilometer,Engine,Max Power,Max Torque,Length,Width,Height,Seating Capacity,...,Fuel Type_LPG,Fuel Type_Petrol,Fuel Type_Petrol + CNG,Fuel Type_Petrol + LPG,Seller Type_Corporate,Seller Type_Individual,Drivetrain_FWD,Drivetrain_RWD,Model_te,Make_te
0,505000,2017,87150.0,1198 cc,87 bhp @ 6000 rpm,109 Nm @ 4500 rpm,3990.0,1680.0,1505.0,5.0,...,0,1,0,0,1,0,1,0,1.702992e+06,6.272787e+05
1,450000,2014,75000.0,1248 cc,74 bhp @ 4000 rpm,190 Nm @ 2000 rpm,3995.0,1695.0,1555.0,5.0,...,0,0,0,0,0,1,1,0,4.067778e+05,5.408743e+05
2,220000,2011,67000.0,1197 cc,79 bhp @ 6000 rpm,112.7619 Nm @ 4000 rpm,3585.0,1595.0,1550.0,5.0,...,0,1,0,0,0,1,1,0,3.035714e+05,7.011824e+05
3,799000,2019,37500.0,1197 cc,82 bhp @ 6000 rpm,113 Nm @ 4200 rpm,3995.0,1745.0,1510.0,5.0,...,0,1,0,0,0,1,1,0,7.750000e+05,1.785524e+06
4,1950000,2018,69000.0,2393 cc,148 bhp @ 3400 rpm,343 Nm @ 1400 rpm,4735.0,1830.0,1795.0,5.0,...,0,0,0,0,0,1,0,1,1.921500e+06,1.677177e+06


In [353]:
# import re
# import numpy as np
# import pandas as pd

# # --- helpers ---
# num_re = r"(\d+(?:\.\d+)?)"   # int or float

# def parse_value_and_rpm(s: str):
#     """
#     Parse strings like:
#       '87 bhp @ 6000 rpm'  -> (87.0, 6000.0)
#       '109 Nm @ 4500 rpm'  -> (109.0, 4500.0)
#       '74 bhp'             -> (74.0, nan)
#       nan / ''             -> (nan, nan)
#     """
#     if pd.isna(s):
#         return (np.nan, np.nan)

#     s = str(s).strip()
#     if not s:
#         return (np.nan, np.nan)

#     # 1) value = first number in the string
#     m_val = re.search(num_re, s)
#     val = float(m_val.group(1)) if m_val else np.nan

#     # 2) rpm = number after '@' (if present)
#     m_rpm = re.search(r"@\s*" + num_re, s)
#     rpm = float(m_rpm.group(1)) if m_rpm else np.nan

#     return (val, rpm)

# --- apply to Max Power ---
df[["MaxPower_value", "MaxPower_rpm"]] = (
    df["Max Power"].apply(parse_value_and_rpm).apply(pd.Series)
)

# --- apply to Max Torque ---
df[["MaxTorque_value", "MaxTorque_rpm"]] = (
    df["Max Torque"].apply(parse_value_and_rpm).apply(pd.Series)
)

# drop original text columns
df.drop(["Max Power", "Max Torque"], axis=1, inplace=True)

# optional: if you want to fill missing rpm values (some rows may not have "@ ... rpm")
# df[["MaxPower_rpm","MaxTorque_rpm"]] = df[["MaxPower_rpm","MaxTorque_rpm"]].fillna(df[["MaxPower_rpm","MaxTorque_rpm"]].median())


In [354]:
# replace nan with median
num_cols = df.select_dtypes(include=['int64', 'float64']).columns
for col in num_cols:
    df[col].fillna(df[col].median(), inplace=True)
    

/tmp/ipykernel_2356366/290604796.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].median(), inplace=True)
/tmp/ipykernel_2356366/290604796.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', tr

In [355]:
df = df.astype({col: int for col in df.select_dtypes("bool").columns})


In [356]:
df.head()

,Price,Year,Kilometer,Engine,Length,Width,Height,Seating Capacity,Fuel Tank Capacity,Owner_ord,...,Seller Type_Corporate,Seller Type_Individual,Drivetrain_FWD,Drivetrain_RWD,Model_te,Make_te,MaxPower_value,MaxPower_rpm,MaxTorque_value,MaxTorque_rpm
0,505000,2017,87150.0,1198 cc,3990.0,1680.0,1505.0,5.0,35.0,1,...,1,0,1,0,1.702992e+06,6.272787e+05,87.0,6000.0,109.0000,4500.0
1,450000,2014,75000.0,1248 cc,3995.0,1695.0,1555.0,5.0,42.0,2,...,0,1,1,0,4.067778e+05,5.408743e+05,74.0,4000.0,190.0000,2000.0
2,220000,2011,67000.0,1197 cc,3585.0,1595.0,1550.0,5.0,35.0,1,...,0,1,1,0,3.035714e+05,7.011824e+05,79.0,6000.0,112.7619,4000.0
3,799000,2019,37500.0,1197 cc,3995.0,1745.0,1510.0,5.0,37.0,1,...,0,1,1,0,7.750000e+05,1.785524e+06,82.0,6000.0,113.0000,4200.0
4,1950000,2018,69000.0,2393 cc,4735.0,1830.0,1795.0,5.0,55.0,1,...,0,1,0,1,1.921500e+06,1.677177e+06,148.0,3400.0,343.0000,1400.0


In [357]:
# import re
# import numpy as np
# import pandas as pd

# def parse_engine_cc(s):
#     """
#     Examples:
#       '1198 cc' -> 1198.0
#       '1498 CC' -> 1498.0
#       '1,198 cc' -> 1198.0
#       NaN / ''  -> NaN
#     """
#     if pd.isna(s):
#         return np.nan
    
#     s = str(s).strip().lower()
#     if not s:
#         return np.nan

#     # remove commas: "1,198" -> "1198"
#     s = s.replace(",", "")

#     m = re.search(r"(\d+(?:\.\d+)?)\s*cc", s)
#     if m:
#         return float(m.group(1))

#     # fallback: just grab first number if "cc" missing
#     m2 = re.search(r"(\d+(?:\.\d+)?)", s)
#     return float(m2.group(1)) if m2 else np.nan


In [358]:
df["Engine_cc"] = df["Engine"].apply(parse_engine_cc)
df.drop(["Engine"], axis=1, inplace=True)


In [359]:
df.head()

,Price,Year,Kilometer,Length,Width,Height,Seating Capacity,Fuel Tank Capacity,Owner_ord,Transmission_bin,...,Seller Type_Individual,Drivetrain_FWD,Drivetrain_RWD,Model_te,Make_te,MaxPower_value,MaxPower_rpm,MaxTorque_value,MaxTorque_rpm,Engine_cc
0,505000,2017,87150.0,3990.0,1680.0,1505.0,5.0,35.0,1,0,...,0,1,0,1.702992e+06,6.272787e+05,87.0,6000.0,109.0000,4500.0,1198.0
1,450000,2014,75000.0,3995.0,1695.0,1555.0,5.0,42.0,2,0,...,1,1,0,4.067778e+05,5.408743e+05,74.0,4000.0,190.0000,2000.0,1248.0
2,220000,2011,67000.0,3585.0,1595.0,1550.0,5.0,35.0,1,0,...,1,1,0,3.035714e+05,7.011824e+05,79.0,6000.0,112.7619,4000.0,1197.0
3,799000,2019,37500.0,3995.0,1745.0,1510.0,5.0,37.0,1,0,...,1,1,0,7.750000e+05,1.785524e+06,82.0,6000.0,113.0000,4200.0,1197.0
4,1950000,2018,69000.0,4735.0,1830.0,1795.0,5.0,55.0,1,0,...,1,0,1,1.921500e+06,1.677177e+06,148.0,3400.0,343.0000,1400.0,2393.0


In [360]:
# check if everything is numeric
df.dtypes

Price                       int64
Year                        int64
Kilometer                 float64
Length                    float64
Width                     float64
Height                    float64
Seating Capacity          float64
Fuel Tank Capacity        float64
Owner_ord                   int64
Transmission_bin            int64
Fuel Type_CNG + CNG         int64
Fuel Type_Diesel            int64
Fuel Type_Electric          int64
Fuel Type_Hybrid            int64
Fuel Type_LPG               int64
Fuel Type_Petrol            int64
Fuel Type_Petrol + CNG      int64
Fuel Type_Petrol + LPG      int64
Seller Type_Corporate       int64
Seller Type_Individual      int64
Drivetrain_FWD              int64
Drivetrain_RWD              int64
Model_te                  float64
Make_te                   float64
MaxPower_value            float64
MaxPower_rpm              float64
MaxTorque_value           float64
MaxTorque_rpm             float64
Engine_cc                 float64
dtype: object

In [361]:
car_data.dtypes

Make                   object
Model                  object
Price                   int64
Year                    int64
Kilometer             float64
Fuel Type              object
Transmission           object
Location               object
Color                  object
Owner                  object
Seller Type            object
Engine                 object
Max Power              object
Max Torque             object
Drivetrain             object
Length                float64
Width                 float64
Height                float64
Seating Capacity      float64
Fuel Tank Capacity    float64
dtype: object

In [362]:
# feature scaling
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
num_cols = df.select_dtypes(include=['int64', 'float64']).columns
num_cols = num_cols.drop("Price")  # exclude target variable
df[num_cols] = scaler.fit_transform(df[num_cols])

In [363]:
# ===== Block B: Save scaler + numeric column order =====
import joblib

scaler_artifacts = {
    "scaler": scaler,
    "num_cols": list(num_cols),   # ORDER MATTERS
}
joblib.dump(scaler_artifacts, "scaler_artifacts.joblib")
print("Saved -> scaler_artifacts.joblib")
print("num_cols (count):", len(num_cols))


Saved -> scaler_artifacts.joblib
num_cols (count): 28


In [364]:
df.head()

,Price,Year,Kilometer,Length,Width,Height,Seating Capacity,Fuel Tank Capacity,Owner_ord,Transmission_bin,...,Seller Type_Individual,Drivetrain_FWD,Drivetrain_RWD,Model_te,Make_te,MaxPower_value,MaxPower_rpm,MaxTorque_value,MaxTorque_rpm,Engine_cc
0,505000,0.170858,1.154298,-0.673941,-0.661496,-0.635706,0.0,-1.149684,-0.439417,-0.904046,...,-5.675357,0.636005,-0.429762,0.036000,-0.619990,-0.638540,1.102804,-0.979049,1.602907,-0.744814
1,450000,-0.721270,0.751534,-0.662465,-0.548810,-0.263001,0.0,-0.673267,1.575637,-0.904046,...,0.176200,0.636005,-0.429762,-0.790393,-0.671320,-0.840837,-0.737370,-0.391922,-0.490492,-0.666465
2,220000,-1.613398,0.486339,-1.603494,-1.300046,-0.300271,0.0,-1.149684,-0.439417,-0.904046,...,0.176200,0.636005,-0.429762,-0.856192,-0.576087,-0.763030,1.102804,-0.951781,1.184228,-0.746381
3,799000,0.765609,-0.491566,-0.662465,-0.173193,-0.598435,0.0,-1.013565,-0.439417,-0.904046,...,0.176200,0.636005,-0.429762,-0.555636,0.068084,-0.716346,1.102804,-0.950055,1.351700,-0.746381
4,1950000,0.468234,0.552638,1.035978,0.465357,1.525984,0.0,0.211507,-0.439417,-0.904046,...,0.176200,-1.572315,2.326871,0.175308,0.003719,0.310699,-1.289423,0.717096,-0.992908,1.127717


In [365]:
df.describe()

,Price,Year,Kilometer,Length,Width,Height,Seating Capacity,Fuel Tank Capacity,Owner_ord,Transmission_bin,...,Seller Type_Individual,Drivetrain_FWD,Drivetrain_RWD,Model_te,Make_te,MaxPower_value,MaxPower_rpm,MaxTorque_value,MaxTorque_rpm,Engine_cc
count,2.059000e+03,2.059000e+03,2.059000e+03,2.059000e+03,2.059000e+03,2.059000e+03,2059.0,2.059000e+03,2.059000e+03,2.059000e+03,...,2.059000e+03,2.059000e+03,2.059000e+03,2.059000e+03,2.059000e+03,2.059000e+03,2.059000e+03,2.059000e+03,2.059000e+03,2.059000e+03
mean,1.702992e+06,1.693794e-14,-6.664573e-17,-1.380365e-17,1.621929e-16,1.928197e-16,0.0,-2.898766e-16,2.933275e-17,-7.592006e-17,...,-7.246915e-17,-1.587419e-16,7.419460e-17,-3.968549e-17,6.556732e-17,1.069783e-16,-2.803866e-16,5.176368e-17,4.141094e-17,9.662553e-17
std,2.419881e+06,1.000243e+00,1.000243e+00,1.000243e+00,1.000243e+00,1.000243e+00,0.0,1.000243e+00,1.000243e+00,1.000243e+00,...,1.000243e+00,1.000243e+00,1.000243e+00,1.000243e+00,1.000243e+00,1.000243e+00,1.000243e+00,1.000243e+00,1.000243e+00,1.000243e+00
min,4.900000e+04,-8.453045e+00,-1.734665e+00,-2.718957e+00,-2.201528e+00,-3.170101e+00,0.0,-2.510875e+00,-2.454470e+00,-9.040460e-01,...,-5.675357e+00,-1.572315e+00,-4.297616e-01,-9.572882e-01,-8.530299e-01,-1.447727e+00,-1.740265e+00,-1.421207e+00,-2.039607e+00,-1.644255e+00
25%,4.849990e+05,-7.212703e-01,-7.733349e-01,-6.831213e-01,-5.488105e-01,-7.847879e-01,0.0,-6.732671e-01,-4.394167e-01,-9.040460e-01,...,1.762004e-01,-1.572315e+00,-4.297616e-01,-6.512035e-01,-6.713204e-01,-7.007851e-01,-7.373703e-01,-9.355582e-01,-8.254358e-01,-7.463806e-01
50%,8.250000e+05,1.708576e-01,-7.719911e-02,1.982326e-01,1.461607e-02,-3.375417e-01,0.0,-1.287909e-01,-4.394167e-01,-9.040460e-01,...,1.762004e-01,6.360048e-01,-4.297616e-01,3.599970e-02,-5.522878e-01,-2.339462e-01,-5.533528e-01,-3.194372e-01,-6.579638e-01,-2.762892e-01
75%,1.925000e+06,7.656095e-01,6.520860e-01,7.720308e-01,4.728696e-01,6.091295e-01,0.0,5.518045e-01,-4.394167e-01,1.106138e+00,...,1.762004e-01,6.360048e-01,-4.297616e-01,3.599970e-02,4.869105e-01,6.374864e-01,1.102804e+00,7.170957e-01,1.184228e+00,5.040624e-01
max,3.500000e+07,1.657737e+00,2.790217e+00,2.950169e+00,3.395175e+00,3.016805e+00,0.0,3.614484e+00,7.620797e+00,1.106138e+00,...,1.762004e-01,6.360048e-01,2.326871e+00,1.170114e+01,1.088868e+01,8.278083e+00,3.173001e+00,3.884682e+00,3.277627e+00,7.707429e+00


In [366]:
df.columns

Index(['Price', 'Year', 'Kilometer', 'Length', 'Width', 'Height',
       'Seating Capacity', 'Fuel Tank Capacity', 'Owner_ord',
       'Transmission_bin', 'Fuel Type_CNG + CNG', 'Fuel Type_Diesel',
       'Fuel Type_Electric', 'Fuel Type_Hybrid', 'Fuel Type_LPG',
       'Fuel Type_Petrol', 'Fuel Type_Petrol + CNG', 'Fuel Type_Petrol + LPG',
       'Seller Type_Corporate', 'Seller Type_Individual', 'Drivetrain_FWD',
       'Drivetrain_RWD', 'Model_te', 'Make_te', 'MaxPower_value',
       'MaxPower_rpm', 'MaxTorque_value', 'MaxTorque_rpm', 'Engine_cc'],
      dtype='object')

In [367]:
# ===== Optional but strongly recommended: Save final processed column order =====
final_feature_cols = [c for c in df.columns if c != "Price"]
joblib.dump({"processed_columns": final_feature_cols}, "processed_columns.joblib")
print("Saved -> processed_columns.joblib")
print("num processed columns:", len(final_feature_cols))
print(final_feature_cols)


Saved -> processed_columns.joblib
num processed columns: 28
['Year', 'Kilometer', 'Length', 'Width', 'Height', 'Seating Capacity', 'Fuel Tank Capacity', 'Owner_ord', 'Transmission_bin', 'Fuel Type_CNG + CNG', 'Fuel Type_Diesel', 'Fuel Type_Electric', 'Fuel Type_Hybrid', 'Fuel Type_LPG', 'Fuel Type_Petrol', 'Fuel Type_Petrol + CNG', 'Fuel Type_Petrol + LPG', 'Seller Type_Corporate', 'Seller Type_Individual', 'Drivetrain_FWD', 'Drivetrain_RWD', 'Model_te', 'Make_te', 'MaxPower_value', 'MaxPower_rpm', 'MaxTorque_value', 'MaxTorque_rpm', 'Engine_cc']


In [368]:
# # save preprocessed data to new CSV
# df.to_csv("../car_details_preprocessed2.csv", index=False)